# **1. Kết nối với google drive**

In [ ]:
# kết nối với gg drive
import os
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/NLP 4+1/Cuối kỳ/pre-trained/')
os.getcwd()

Mounted at /content/drive


'/content/drive/MyDrive/NLP 4+1/Cuối kỳ/pre-trained'

# **2. Install và import các thư viện cần thiết**

In [ ]:
!pip install transformers --quiet
!pip install keras_preprocessing --quiet
!pip install pyvi --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 62.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 62.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 80.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00


In [ ]:
import os
import re
import time
import torch
import pickle
import numpy as np
import pandas as pd

from pyvi import ViTokenizer
from keras.models import load_model
from keras_preprocessing.sequence import pad_sequences
# from transformers import GPT2LMHeadModel, BertLMHeadModel

import nltk
nltk.download('punkt')

from nltk import word_tokenize

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


# **3. Set các biến cục bộ và load dữ liệu**

In [ ]:
def preprocess(text):
    text = text.lower()                                                 # chuyển về chữ thường
    text = re.sub(r'[!“”"’#$%&\()*+,./:;<=>?@[\]^_`{|}~]', "", text)    # loại bỏ kí tự đặt biệt
    text = ViTokenizer.tokenize(text)                                   # word segmentation
    return text

In [ ]:
# loading
def load_file(path):
    with open(path, 'rb') as handle:
        data = pickle.load(handle)
    return data

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# **4. Load pre-trained cho các model**

## ***4.1 Model GPT***

In [ ]:
tokenizer_GPT = load_file('./GPT/tokenizer.pickle')
model_GPT = load_file('./GPT/GPT2LMHeadModel.pickle')

In [ ]:
model_GPT.resize_token_embeddings(len(tokenizer_GPT))
model_GPT = model_GPT.to(device)
if device == 'cpu':
    model_GPT.load_state_dict(torch.load('./GPT/model_GPT.pt', map_location ='cpu'))
else:
    model_GPT.load_state_dict(torch.load('./GPT/model_GPT.pt'))

In [ ]:
def GPT_answer(inp, tokenizer=tokenizer_GPT, model=model_GPT):
    inp = '<s> '+ preprocess(inp) +" <bot>: "
    inp = tokenizer(inp, return_tensors="pt")
    X = inp["input_ids"].to(device)
    mask = inp["attention_mask"].to(device)
    output = model.generate(X, attention_mask=mask, max_length=32, pad_token_id=0, eos_token_id=50256)
    output = tokenizer.decode(output[0])
    try:
        result = re.search(r'<bot>:(.*?)</s>', output).group(1).strip()
    except:
        result = re.search(r'<bot>:\s*(.*)', output).group(1).strip()
    return result

In [ ]:
GPT_answer("bạn có người yêu chưa?")

'mình chưa có người_yêu bạn nhé'

## ***4.2 Model LSTM, BiLSTM***

In [ ]:
# LSTM (100 epoch)
tokenizer_LSTM_100 = load_file( './LSTM (100 epoch)/tokenizer.pickle')
enc_model_LSTM_100 = load_model('./LSTM (100 epoch)/enc_model.h5')
dec_model_LSTM_100 = load_model('./LSTM (100 epoch)/dec_model.h5')

# LSTM (200 epoch)
tokenizer_LSTM_200 = load_file( './LSTM (200 epoch)/tokenizer.pickle')
enc_model_LSTM_200 = load_model('./LSTM (200 epoch)/enc_model.h5')
dec_model_LSTM_200 = load_model('./LSTM (200 epoch)/dec_model.h5')

# LSTM + Attention (100 epoch)
tokenizer_LSTM_Attention_100 = load_file( './LSTM + Attention (100 epoch)/tokenizer.pickle')
enc_model_LSTM_Attention_100 = load_model('./LSTM + Attention (100 epoch)/enc_model.h5')
dec_model_LSTM_Attention_100 = load_model('./LSTM + Attention (100 epoch)/dec_model.h5')

# LSTM + Attention (200 epoch)
tokenizer_LSTM_Attention_200 = load_file( './LSTM + Attention (200 epoch)/tokenizer.pickle')
enc_model_LSTM_Attention_200 = load_model('./LSTM + Attention (200 epoch)/enc_model.h5')
dec_model_LSTM_Attention_200 = load_model('./LSTM + Attention (200 epoch)/dec_model.h5')

In [ ]:
# BiLSTM (100 epoch)
tokenizer_BiLSTM_100 = load_file( './BiLSTM (100 epoch)/tokenizer.pickle')
enc_model_BiLSTM_100 = load_model('./BiLSTM (100 epoch)/enc_model.h5')
dec_model_BiLSTM_100 = load_model('./BiLSTM (100 epoch)/dec_model.h5')

# BiLSTM (200 epoch)
tokenizer_BiLSTM_200 = load_file( './BiLSTM (200 epoch)/tokenizer.pickle')
enc_model_BiLSTM_200 = load_model('./BiLSTM (200 epoch)/enc_model.h5')
dec_model_BiLSTM_200 = load_model('./BiLSTM (200 epoch)/dec_model.h5')

# BiLSTM + Attention (100 epoch)
tokenizer_BiLSTM_Attention_100 = load_file( './BiLSTM + Attention (100 epoch)/tokenizer.pickle')
enc_model_BiLSTM_Attention_100 = load_model('./BiLSTM + Attention (100 epoch)/enc_model.h5')
dec_model_BiLSTM_Attention_100 = load_model('./BiLSTM + Attention (100 epoch)/dec_model.h5')

# BiLSTM + Attention (200 epoch)
tokenizer_BiLSTM_Attention_200 = load_file( './BiLSTM + Attention (200 epoch)/tokenizer.pickle')
enc_model_BiLSTM_Attention_200 = load_model('./BiLSTM + Attention (200 epoch)/enc_model.h5')
dec_model_BiLSTM_Attention_200 = load_model('./BiLSTM + Attention (200 epoch)/dec_model.h5')

In [ ]:
MAXLEN_QUESTION = 15
MAXLEN_ANSWER = 15
def str_to_tokens(sentence, tokenizer):
    words = preprocess(sentence)
    tokens_list = tokenizer.texts_to_sequences([words])
    return pad_sequences(tokens_list, maxlen=MAXLEN_QUESTION,padding='post')
    
def LSTM_BiLSTM_answer(question, type_model, with_epochs):
    if with_epochs == 100:
        if type_model == 'LSTM':
            tokenizer, enc_model, dec_model = tokenizer_LSTM_100, enc_model_LSTM_100, dec_model_LSTM_100
        elif type_model == 'BiLSTM':
            tokenizer, enc_model, dec_model = tokenizer_BiLSTM_100, enc_model_BiLSTM_100, dec_model_BiLSTM_100
    elif with_epochs == 200:
        if type_model == 'LSTM':
            tokenizer, enc_model, dec_model = tokenizer_LSTM_200, enc_model_LSTM_200, dec_model_LSTM_200
        elif type_model == 'BiLSTM':
            tokenizer, enc_model, dec_model = tokenizer_BiLSTM_200, enc_model_BiLSTM_200, dec_model_BiLSTM_200

    states_values = enc_model.predict(str_to_tokens(question, tokenizer), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['<s>']
    stop_condition = False
    generated_answer = ''
    while not stop_condition:
        dec_outputs, h, c = dec_model.predict([empty_target_seq]+states_values, verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    generated_answer += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(generated_answer.split()) > MAXLEN_ANSWER:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        states_values = [h,c]

    return generated_answer.strip()

def LSTM_BiLSTM_Attention_answer(question, type_model, with_epochs):
    if with_epochs == 100:
        if type_model == 'LSTM':
            tokenizer, enc_model, dec_model = tokenizer_LSTM_Attention_100, enc_model_LSTM_Attention_100, dec_model_LSTM_Attention_100
        elif type_model == 'BiLSTM':
            tokenizer, enc_model, dec_model = tokenizer_BiLSTM_Attention_100, enc_model_BiLSTM_Attention_100, dec_model_BiLSTM_Attention_100
    elif with_epochs == 200:
        if type_model == 'LSTM':
            tokenizer, enc_model, dec_model = tokenizer_LSTM_Attention_200, enc_model_LSTM_Attention_200, dec_model_LSTM_Attention_200
        elif type_model == 'BiLSTM':
            tokenizer, enc_model, dec_model = tokenizer_BiLSTM_Attention_200, enc_model_BiLSTM_Attention_200, dec_model_BiLSTM_Attention_200

    values,h,c = enc_model.predict(str_to_tokens(question, tokenizer), verbose=0)
    empty_target_seq = np.zeros((1,1))
    empty_target_seq[0,0] = tokenizer.word_index['<s>']
    stop_condition = False
    generated_answer = ''

    while not stop_condition:
        dec_outputs, h_state,c_state = dec_model.predict([empty_target_seq]+[values,h,c], verbose=0)
        sampled_word_index = np.argmax(dec_outputs[0,-1, :])
        sampled_word = None
        for word, index in tokenizer.word_index.items():
            if sampled_word_index == index:
                if word != '</s>':
                    generated_answer += f'{word} '
                sampled_word = word

        if sampled_word == '</s>' or len(generated_answer.split()) > MAXLEN_ANSWER:
            stop_condition = True
        empty_target_seq = np.zeros((1,1))
        empty_target_seq[0,0] = sampled_word_index
        h,c = h_state,c_state

    return generated_answer.strip()

In [ ]:
print('LSTM (100 epochs):', LSTM_BiLSTM_answer('bạn có người yêu chưa?', type_model='LSTM', with_epochs=100))
print('LSTM (200 epochs):', LSTM_BiLSTM_answer('bạn có người yêu chưa?', type_model='LSTM', with_epochs=200))
print('LSTM + Attention (100 epochs):', LSTM_BiLSTM_Attention_answer('bạn có người yêu chưa?', type_model='LSTM', with_epochs=100))
print('LSTM + Attention (200 epochs):', LSTM_BiLSTM_Attention_answer('bạn có người yêu chưa?', type_model='LSTM', with_epochs=200))

LSTM (100 epochs): có
LSTM (200 epochs): tôi có người_yêu rồi
LSTM + Attention (100 epochs): có
LSTM + Attention (200 epochs): vẫn chưa


In [ ]:
print('BiLSTM (100 epochs):', LSTM_BiLSTM_answer('bạn có người yêu chưa?', type_model='BiLSTM', with_epochs=100))
print('BiLSTM (200 epochs):', LSTM_BiLSTM_answer('bạn có người yêu chưa?', type_model='BiLSTM', with_epochs=200))
print('BiLSTM + Attention (100 epochs):', LSTM_BiLSTM_Attention_answer('bạn có người yêu chưa?', type_model='BiLSTM', with_epochs=100))
print('BiLSTM + Attention (200 epochs):', LSTM_BiLSTM_Attention_answer('bạn có người yêu chưa?', type_model='BiLSTM', with_epochs=200))

BiLSTM (100 epochs): có
BiLSTM (200 epochs): mình chưa có người_yêu
BiLSTM + Attention (100 epochs): mình không thích gì thế_giới lên kinh_tế
BiLSTM + Attention (200 epochs): mình có người_yêu rồi


# **5. Chat với các model**

In [ ]:
def chatbot_all(inp):
    print('USER                      :', inp)
    print('(LSTM 100)                :', LSTM_BiLSTM_answer(inp, type_model='LSTM', with_epochs=100)) 
    print('(LSTM 200)                :', LSTM_BiLSTM_answer(inp, type_model='LSTM', with_epochs=200))
    print('(LSTM + Attention 100)    :', LSTM_BiLSTM_Attention_answer(inp, type_model='LSTM', with_epochs=100)) 
    print('(LSTM + Attention 200)    :', LSTM_BiLSTM_Attention_answer(inp, type_model='LSTM', with_epochs=200)) 
    print('(BiLSTM 100)              :', LSTM_BiLSTM_answer(inp, type_model='BiLSTM', with_epochs=100)) 
    print('(BiLSTM 200)              :', LSTM_BiLSTM_answer(inp, type_model='BiLSTM', with_epochs=200))
    print('(BiLSTM + Attention 100)  :', LSTM_BiLSTM_Attention_answer(inp, type_model='BiLSTM', with_epochs=100)) 
    print('(BiLSTM + Attention 200)  :', LSTM_BiLSTM_Attention_answer(inp, type_model='BiLSTM', with_epochs=200))
    print('(GPT)                     :', GPT_answer(inp)) 
      
chatbot_all('bạn có người yêu chưa?')

USER                      : bạn có người yêu chưa?
(LSTM 100)                : có
(LSTM 200)                : tôi có người_yêu rồi
(LSTM + Attention 100)    : có
(LSTM + Attention 200)    : vẫn chưa
(BiLSTM 100)              : có
(BiLSTM 200)              : mình chưa có người_yêu
(BiLSTM + Attention 100)  : mình không thích gì thế_giới lên kinh_tế
(BiLSTM + Attention 200)  : mình có người_yêu rồi
(GPT)                     : mình chưa có người_yêu bạn nhé


In [ ]:
def chatbot_with_model(name_model):
    print('bot  : Xin chào! Tôi có thể giúp gì cho bạn?')
    while True:
        inp = input('user : ')
        if inp == 'tạm biệt':
            print('bot  : Hẹn gặp lại bạn lần sau!')
            break
            
        if name_model == 'LSTM_100':
            print('bot  :', LSTM_BiLSTM_answer(inp, type_model='LSTM', with_epochs=100))
        elif name_model == 'LSTM_200':
            print('bot  :', LSTM_BiLSTM_answer(inp, type_model='LSTM', with_epochs=200))
        elif name_model == 'LSTM_Attention_100':
            print('bot  :', LSTM_BiLSTM_Attention_answer(inp, type_model='LSTM', with_epochs=100))
        elif name_model == 'LSTM_Attention_200':
            print('bot  :', LSTM_BiLSTM_Attention_answer(inp, type_model='LSTM', with_epochs=200))
        elif name_model == 'BiLSTM_100':
            print('bot  :', LSTM_BiLSTM_answer(inp, type_model='BiLSTM', with_epochs=100))
        elif name_model == 'BiLSTM_200':
            print('bot  :', LSTM_BiLSTM_answer(inp, type_model='BiLSTM', with_epochs=200))
        elif name_model == 'BiLSTM_Attention_100':
            print('bot  :', LSTM_BiLSTM_Attention_answer(inp, type_model='BiLSTM', with_epochs=100))
        elif name_model == 'BiLSTM_Attention_200':
            print('bot  :', LSTM_BiLSTM_Attention_answer(inp, type_model='BiLSTM', with_epochs=200))
        elif name_model == 'GPT':
            print('bot  :', GPT_answer(inp))

chatbot_with_model(name_model='BiLSTM_Attention_200')

bot  : Xin chào! Tôi có thể giúp gì cho bạn?
user : bạn học ngành gì
bot  : học ngôn_ngữ còn bạn
user : bạn ở đâu
bot  : mình ở quận 8
user : tạm biệt
bot  : Hẹn gặp lại bạn lần sau!
